In [4]:
conda install --no-deps -c fastai fastai

Solving environment: done

## Package Plan ##

  environment location: /home/ubuntu/anaconda3

  added / updated specs:
    - fastai


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    fastai-1.0.61              |                1         177 KB  fastai
    ------------------------------------------------------------
                                           Total:         177 KB

The following NEW packages will be INSTALLED:

  fastai             fastai/noarch::fastai-1.0.61-1



fastai-1.0.61        | 177 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [1]:
import aerospike
config_aerospike = {'hosts': [('35.228.136.58', 3000)]}
client = aerospike.client(config_aerospike).connect()
bert_data_key = ("ids_to_validate", "data", "bert_tags")

In [2]:
from fastai import *
from fastai.vision import *

In [3]:
from tagger import config, train_model, stringConverter
from tagger import predict

/home/ubuntu/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: thinc.extra.search.Beam size changed, may indicate binary incompatibility. Expected 112 from C header, got 120 from PyObject
  return f(*args, **kwds)


In [4]:
argsIndustries = {
    "labels": client.get(bert_data_key)[-1]["gutenIndustries"],
    "model": load_learner("models", "guten_industries_P3_v1.pkl"),
}

In [5]:
argsShifts = {
    "labels": client.get(bert_data_key)[-1]["gutenShifts"],
    "model": load_learner("models", "guten_shifts_P3_v1.pkl"),
}

In [6]:
import bson
import pymongo
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags



In [7]:
from datetime import datetime

In [8]:
from tqdm import tqdm

In [9]:
def updateDB(_id, shiftsPredict, industriesPredict, shifts, industries):
    try:
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(_id)},
                                        {"$set":{
                                                 "industries_from_bert": industriesPredict,
                                                 "shifts_from_bert": shiftsPredict,
                                                 "before_shifts_industries":{
                                                     "update_date": datetime.now(),
                                                    "industries_from_bert": industries,
                                                     "shifts_from_bert":shifts,
                                                 }
                                        }})    
    

In [12]:
db = db_client["tweets_pipeline_v2"].find({"$or":[{"status": "reserved"},
#                                                {"status": "validated"}
                                                 ],
                                          }, no_cursor_timeout=True)
for i in tqdm(db, total = db.count()):
    try:
        industriesPredict = predict(stringConverter(i["extracted_text"], use_mister_proper=True), argsIndustries)
        shiftsPredict = predict(stringConverter(i["extracted_text"], use_mister_proper=True), argsShifts)
        industries = []
        shifts = []
        if "industries_from_bert" in i:
            industries = i["industries_from_bert"]
        if "shifts_from_bert" in i:
            shifts = i['shifts_from_bert']
        _id = str(i["_id"])
        updateDB(_id, shiftsPredict=shiftsPredict, industriesPredict=industriesPredict, shifts = shifts, industries = industries)
    except Exception:
        pass
db.close()    

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """
100%|██████████| 13931/13931 [30:20<00:00,  7.65it/s] 
